## Z-test for Sessions with and without has_video field

In [2]:
# Install requirements.txt if not already installed
import sys
import subprocess

try:
    import os
    if os.path.exists('requirements.txt'):
        print("Installing packages from requirements.txt ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    else:
        print("requirements.txt not found.")
except Exception as e:
    print(f"Error installing requirements: {e}")

Installing packages from requirements.txt ...



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [141]:
#Params
#do you need to download an existing data?
download = False

START_DATE = '2025-02-01'
END_DATE = '2025-08-01'

#OR Relative date range
# days_back_for_sql = 30  # Number of days back from the end date for SQL
# START_DATE = (end_date - timedelta(days=days_back_for_sql)).strftime('%Y-%m-%d')
# END_DATE = end_date.strftime('%Y-%m-%d')

# Other query parameters
REVIEW_TYPE = ['random_sample', 'customer_reporting_1', 'authentication_product'] #['random_sample', 'customer_reporting_1', 'authentication_product']
REVIEW_TYPE_FOR_QUERY = ', '.join(["'{}'".format(review_type) for review_type in REVIEW_TYPE])

# for Redshift connections
REDSHIFT_HOST = "wg-serverless-1.875431798560.eu-west-1.redshift-serverless.amazonaws.com"
REDSHIFT_DATABASE = "vaire"
REDSHIFT_USER = os.environ.get("REDSHIFT_USER")
REDSHIFT_PASSWORD = os.environ.get("REDSHIFT_PASSWORD")
REDSHIFT_PORT = 5439

## ready file if any
file_path = 'sample_data.csv'

In [142]:
#Redshift Connector 

import redshift_connector
import pandas as pd
import logging
import os

# Configure logging for better error visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def get_redshift_dataframe(
    sql_query: str,
    host: str,
    database: str,
    user: str,
    password: str,
    port: int = 5439 # Default Redshift port
) -> pd.DataFrame:
    """
    Connects to Amazon Redshift, executes a SQL query, and returns the result
    as a pandas DataFrame.

    Args:
        sql_query (str): The SQL query string to execute.
        host (str): The Redshift cluster endpoint (e.g., 'your-cluster.xxxx.region.redshift.amazonaws.com').
        database (str): The name of the database to connect to.
        user (str): The username for Redshift authentication.
        password (str): The password for Redshift authentication.
        port (int): The port number for the Redshift connection (default is 5439).

    Returns:
        pd.DataFrame: A pandas DataFrame containing the query results.
                      Returns an empty DataFrame if an error occurs or no data is found.

    Raises:
        Exception: Re-raises any connection or query execution errors after logging.
    """
    conn = None # Initialize connection to None
    try:
        logging.info(f"Attempting to connect to Redshift database: {database}@{host}:{port} with user: {user}")
        conn = redshift_connector.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
        logging.info("Successfully connected to Redshift.")

        logging.info("Executing SQL query...")
        # Use pandas read_sql for direct DataFrame conversion
        df = pd.read_sql(sql_query, conn)
        logging.info(f"Query executed successfully. Fetched {len(df)} rows.")

        return df

    except redshift_connector.Error as e:
        logging.error(f"Redshift connection or query error: {e}")
        return pd.DataFrame() # Return empty DataFrame on specific Redshift errors
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        # Optionally re-raise the exception if you want calling code to handle it
        raise
    finally:
        if conn:
            logging.info("Closing Redshift connection.")
            conn.close()
        else:
            logging.warning("Connection was not established, so nothing to close.")

In [143]:
query = f"""
WITH verification_sessions AS (
SELECT 
verification_session_uuid
, vs.shard_key
, vs.has_video
, vs.verification_session_tool_name
, vs.service_agreement_stage
, vs.primary_product
, CASE
    WHEN vs.primary_product IN ('IDV', 'Document Verification', 'AIC') THEN 'IDV'
    WHEN vs.primary_product LIKE 'Full Auto%' THEN 'Full Auto'
    WHEN vs.primary_product LIKE '%Biometric%' OR vs.primary_product IN ('Face Match', 'Age Estimation') THEN 'Biometric Solutions'
    WHEN vs.primary_product = 'Unknown' THEN 'Unknown'
    ELSE 'Data Verification'
END as product_portfolio
FROM analytics_mart_kpi.mart_kpi__verification_session_metrics vs
WHERE date(vs.verified_at) >= CAST('{START_DATE}' AS DATE)
AND date(vs.verified_at) <= CAST('{END_DATE}' AS DATE)
)
, data as (
SELECT 
qa.verification_session_uuid
, qa.shard_key
, qa.verification_session_uuid as qa_verification_session_uuid
, qa.qa_review_type
, qa.flag_session_has_decision_error
, qa.flag_session_has_critical_approve_error
, qa.flag_session_has_critical_decline_error
, qa.flag_session_has_extraction_error
, qa.flag_session_has_critical_error
, qa.is_session_approved_fraud
, qa.is_session_fraud
, qa.qa_decision
, qa.original_decision
, qa.qa_decision = 'declined' as qa_declinable
, qa.qa_decision = 'approved' as qa_approvable
--verification
, vs.has_video
, vs.primary_product
, vs.product_portfolio
, vs.service_agreement_stage
, vs.verification_session_tool_name
FROM analytics_mart_automation.mart_automation__verification_session_qa_review qa
LEFT JOIN verification_sessions vs
    ON vs.verification_session_uuid = qa.verification_session_uuid
    AND vs.shard_key = qa.shard_key
WHERE TRUE
AND qa.verified_date >= CAST('{START_DATE}' AS DATE)
AND qa.verified_date <= CAST('{END_DATE}' AS DATE)
AND qa.qa_review_type IN ({REVIEW_TYPE_FOR_QUERY}) --('random_sample', 'customer_reporting_1', 'authentication_product')
AND vs.service_agreement_stage NOT IN ('test', 'test_low')
)
, aggregated_data as (
SELECT
a.primary_product
, a.product_portfolio
, a.has_video
, a.verification_session_tool_name
--from QA 
, COALESCE(COUNT(DISTINCT a.qa_verification_session_uuid),0) as qa_reviewed
, SUM(a.flag_session_has_decision_error::int) as decision_error_sessions
, SUM(a.flag_session_has_critical_error::int) as critical_error_sessions
, SUM(a.flag_session_has_extraction_error::int) as extraction_error_sessions
, SUM(a.flag_session_has_critical_approve_error::int) as false_approves
, SUM(a.flag_session_has_critical_decline_error::int) as false_declines
, SUM(a.is_session_approved_fraud::int) as missed_fraud_sessions
, SUM(a.qa_declinable::int) as declinable_sessions
, SUM(a.qa_approvable::int) as approvable_sessions
, SUM(a.is_session_fraud::int) as fraud_sessions
, 1.00 * decision_error_sessions / NULLIF(qa_reviewed, 0) as decision_error_rate
, 1.00 * critical_error_sessions / NULLIF(qa_reviewed, 0) as critical_error_rate
, 1.00 * extraction_error_sessions / NULLIF(qa_reviewed, 0) as extraction_error_rate
, 1.00 * false_approves / NULLIF(declinable_sessions, 0) as false_approve_rate
, 1.00 * false_declines / NULLIF(approvable_sessions, 0) as false_rejection_rate
, 1.00 * missed_fraud_sessions / NULLIF(fraud_sessions, 0) as missed_fraud_rate
, 1.00 * declinable_sessions / NULLIF(qa_reviewed, 0) as decline_rate_in_qa
, 1.00 * approvable_sessions / NULLIF(qa_reviewed, 0) as approve_rate_in_qa
, 1.00 * fraud_sessions / NULLIF(qa_reviewed, 0) as fraud_rate_in_qa
--, sum(qa_reviewed) over () as total_qa_reviewed
--, 1.00 * qa_reviewed / NULLIF(total_qa_reviewed,0) as proportion_of_qa_sample
from data a
group by 1,2,3,4
)
SELECT *
FROM aggregated_data
"""

In [144]:
if download:
    df = pd.read_csv(file_path)
else:
    df = get_redshift_dataframe(
                sql_query=query, # Pass the multi-line query variable here
                host=REDSHIFT_HOST,
                database=REDSHIFT_DATABASE,
                user=REDSHIFT_USER,
                password=REDSHIFT_PASSWORD,
                port=REDSHIFT_PORT
            )

    if not df.empty:
        # Save the DataFrame to a CSV file
        df.to_csv(file_path, index=False)
        print("\nSuccessfully fetched data from Redshift:")
        print(df.head())
        print(f"\nDataFrame shape: {df.shape}")
    else:
        print("\nNo data fetched or an error occurred.")

2025-08-07 13:00:09,075 - INFO - Attempting to connect to Redshift database: vaire@wg-serverless-1.875431798560.eu-west-1.redshift-serverless.amazonaws.com:5439 with user: gadir_mamedov
2025-08-07 13:00:09,551 - INFO - Successfully connected to Redshift.
2025-08-07 13:00:09,552 - INFO - Executing SQL query...
/var/folders/21/4tgxh7dn3vjdh5xpfj_mh_pr0000gn/T/ipykernel_26163/1392376839.py:52: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

2025-08-07 13:02:19,306 - INFO - Query executed successfully. Fetched 32 rows.
2025-08-07 13:02:19,307 - INFO - Closing Redshift connection.



Successfully fetched data from Redshift:
              primary_product    product_portfolio  has_video  \
0                         IDV                  IDV      False   
1    Biometric Authentication  Biometric Solutions       True   
2            Proof of Address    Data Verification      False   
3                     Unknown              Unknown      False   
4  Biometric Passive Liveness  Biometric Solutions       True   

  verification_session_tool_name  qa_reviewed  decision_error_sessions  \
0                      full_auto        43300                     1737   
1                      full_auto         8919                      207   
2                      full_auto            9                        0   
3                      full_auto         3981                      306   
4                      full_auto       105002                     1808   

   critical_error_sessions  extraction_error_sessions  false_approves  \
0                      185                      3

In [174]:
df_original = df.copy()

In [175]:
df = df_original.copy()

In [176]:
df=df[df['primary_product'].isin(['Document Verification', 
                                  'Biometric Passive Liveness', 'IDV',
                                  'Biometric Authentication',
                                   'Full Auto IDV (Selfie & Document)',
                                   'Full Auto IDV (Document Only)'])]

In [177]:
df = df[df['verification_session_tool_name'] == 'verification_tool']

In [178]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Z-score for a 95% confidence interval
z_score = 1.96

# 2. Define the metrics to analyze
# Each metric requires a 'successes' column (x) and a 'trials' column (n)
metrics = {
    'missed_fraud_rate': {
        'successes': 'missed_fraud_sessions',
        'trials': 'fraud_sessions'
    },
    'false_approve_rate': {
        'successes': 'false_approves',
        'trials': 'declinable_sessions'
    },
    'false_rejection_rate': {
        'successes': 'false_declines',
        'trials': 'approvable_sessions'
    },
    'critical_error_rate': {
        'successes': 'critical_error_sessions',
        'trials': 'qa_reviewed'
    },
    'extraction_error_rate': {
        'successes': 'extraction_error_sessions',
        'trials': 'qa_reviewed'
    },
    # 'decision_error_rate': {
    #     'successes': 'decision_error_sessions',
    #     'trials': 'qa_reviewed'
    # }
}

# 3. Define the dimensions for analysis
test_dimension = 'has_video'

dimensions = [
    #['total'],
    ['primary_product'],
   # ['product_portfolio'],
  #['primary_product', 'verification_session_tool_name']
]

# List to collect all results
results_list = []

# 4. Perform the analysis
for dim in dimensions:
    print(f"===========================================================")
    print(f"Analyzing by dimension: {', '.join(dim)}")
    print(f"===========================================================")
    
    # Group the data by the current dimension and has_video
    grouped_data = df.groupby(dim + [test_dimension]).agg({
        'missed_fraud_sessions': 'sum',
        'fraud_sessions': 'sum',
        'false_approves': 'sum',
        'critical_error_sessions': 'sum',
        'extraction_error_sessions': 'sum',
        'decision_error_sessions': 'sum',
        'declinable_sessions': 'sum',
        'false_declines': 'sum',
        'approvable_sessions': 'sum',
        'qa_reviewed': 'sum'
    }).reset_index()

    # Iterate through each unique group within the dimension
    for (group_name,), group_df in grouped_data.groupby(dim):
        
        # Check if both has_video groups exist
        if len(group_df) < 2:
            continue
            
        for metric_name, cols in metrics.items():
            successes_col = cols['successes']
            trials_col = cols['trials']
            
            # Extract data for the two groups
            try:
                x1 = group_df[group_df[test_dimension] == True][successes_col].iloc[0]
                n1 = group_df[group_df[test_dimension] == True][trials_col].iloc[0]
                x2 = group_df[group_df[test_dimension] == False][successes_col].iloc[0]
                n2 = group_df[group_df[test_dimension] == False][trials_col].iloc[0]
            except IndexError:
                continue

            # Ensure there are enough trials for the test
            if n1 <= 0 or n2 <= 0:
                continue

            # Calculate rates
            p1 = x1 / n1
            p2 = x2 / n2
            delta = p1 - p2

            # Calculate error margins and confidence ranges for both groups
            margin_1 = z_score * np.sqrt((p1 * (1 - p1)) / n1)
            range_1 = f"[{p1 - margin_1:.4f}, {p1 + margin_1:.4f}]"

            margin_2 = z_score * np.sqrt((p2 * (1 - p2)) / n2)
            range_2 = f"[{p2 - margin_2:.4f}, {p2 + margin_2:.4f}]"

            # Perform the Z-test for proportions
            try:
                stat, p_value = proportions_ztest([x1, x2], [n1, n2], alternative='two-sided')
            except ZeroDivisionError:
                stat, p_value = float('nan'), float('nan')

            # Determine significance (alpha = 0.05)
            is_significant = p_value < 0.05
            
            # Append results to the list
            results_list.append({
                'Dimension': ', '.join(dim),
                'Group': group_name if isinstance(group_name, str) else str(group_name),
                'Metric': metric_name,
                'Delta': delta,
                'P-Value': p_value,
                'Significant': is_significant,
                'Rate_has_video_True': p1,
                'Rate_has_video_False': p2,
                'Margin_has_video_True': margin_1,
                'Range_has_video_True': range_1,
                'Margin_has_video_False': margin_2,
                'Range_has_video_False': range_2,
                #'Z-Score': stat,
            })

# 5. Create a DataFrame from the results and save to a file
if results_list:
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('statistical_analysis_results.csv', index=False)
    
    print("-----------------------------------------------------------")
    print("Analysis complete. Results table:")
    print("-----------------------------------------------------------")
    print(results_df)
    print("\nResults have been saved to 'statistical_analysis_results.csv'.")
else:
    print("No statistical tests were performed. Please check your data for valid groups.")

Analyzing by dimension: primary_product
-----------------------------------------------------------
Analysis complete. Results table:
-----------------------------------------------------------
          Dimension                       Group                 Metric  \
0   primary_product    Biometric Authentication      missed_fraud_rate   
1   primary_product    Biometric Authentication     false_approve_rate   
2   primary_product    Biometric Authentication   false_rejection_rate   
3   primary_product    Biometric Authentication    critical_error_rate   
4   primary_product    Biometric Authentication  extraction_error_rate   
5   primary_product  Biometric Passive Liveness      missed_fraud_rate   
6   primary_product  Biometric Passive Liveness     false_approve_rate   
7   primary_product  Biometric Passive Liveness   false_rejection_rate   
8   primary_product  Biometric Passive Liveness    critical_error_rate   
9   primary_product  Biometric Passive Liveness  extraction_error_

/Users/gadirmamedov/python_notebooks/.coding-venv/lib/python3.13/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning:

invalid value encountered in scalar divide



In [179]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# 1. Load the data from the analysis script's output
try:
    df_results = pd.read_csv('statistical_analysis_results.csv')
    print("Results data loaded successfully.\n")
except FileNotFoundError:
    print("Error: 'statistical_analysis_results.csv' not found.")
    print("Please run the statistical analysis script first to generate this file.")
    exit()

# 2. Preprocess data for plotting
df_plot = df_results.copy()
df_plot['Group_Label'] = df_plot['Group'] #df_plot['Dimension'] + ': ' + df_plot['Group']

# Create a list of unique metrics
metrics_to_plot = df_results['Metric'].unique()

# 3. Generate a plot for each metric using Plotly
for metric in metrics_to_plot:
    subset = df_plot[df_plot['Metric'] == metric].copy()

    if subset.empty:
        continue

    subset = subset.sort_values('Group_Label')
    
    fig = go.Figure()

    # Define a single color for each has_video group
    color_true = 'green'
    color_false = 'salmon'

    # Define marker shapes for significance
    # A circle for non-significant, and a different shape (e.g., a star) for significant
    subset['marker_shape'] = subset['Significant'].apply(lambda x: 'star' if x else 'circle')

    # Plot for has_video = True
    fig.add_trace(go.Scatter(
        x=subset['Group_Label'],
        y=subset['Rate_has_video_True'],
        mode='markers',
        marker=dict(
            size=10,
            color=color_true,
            symbol=subset['marker_shape']
        ),
        name='has_video = True',
        error_y=dict(
            type='data',
            symmetric=True,
            array=subset['Margin_has_video_True'],
            visible=True,
            thickness=2,
            width=5
        )
    ))

    # Plot for has_video = False
    fig.add_trace(go.Scatter(
        x=subset['Group_Label'],
        y=subset['Rate_has_video_False'],
        mode='markers',
        marker=dict(
            size=10,
            color=color_false,
            symbol=subset['marker_shape']
        ),
        name='has_video = False',
        error_y=dict(
            type='data',
            symmetric=True,
            array=subset['Margin_has_video_False'],
            visible=True,
            thickness=2,
            width=5
        )
    ))
    
    # Final plot adjustments
    fig.update_layout(
        title=f'Metric: {metric} with 95% Confidence Intervals',
        xaxis_title='Dimension Group',
        yaxis_title='Rate',
        legend_title='Has Video?',
        font=dict(size=12),
        height=600,
        width=800,
        margin=dict(l=50, r=20, b=100, t=100)
    )

    # Add a custom legend entry for significance
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='grey', symbol='star'),
        name='Significant'
    ))
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color='grey', symbol='circle'),
        name='Not Significant'
    ))

    fig.show()

print("Plots generated successfully.")

Results data loaded successfully.



Plots generated successfully.


In [180]:
results_df['Delta, %'] = round(results_df['Delta']*100,2) # Convert to percentage
results_df['P-Value'] = round(results_df['P-Value'],3)  # Round P-Value for better readability
results_df['Rate_has_video_True, %'] = round(results_df['Rate_has_video_True']*100, 2).astype(str) + '±' + round(results_df['Margin_has_video_True']*100, 2).astype(str)
results_df['Rate_has_video_False, %'] = round(results_df['Rate_has_video_False']*100, 2).astype(str) + '±' + round(results_df['Margin_has_video_False']*100, 2).astype(str)
results_df[['Group', 'Metric', 'Delta, %', 'P-Value', 'Significant', 'Rate_has_video_True, %', 'Rate_has_video_False, %']]

,Group,Metric,"Delta, %",P-Value,Significant,"Rate_has_video_True, %","Rate_has_video_False, %"
0,Biometric Authentication,missed_fraud_rate,0.00,NaN,False,0.0±0.0,0.0±0.0
1,Biometric Authentication,false_approve_rate,0.00,NaN,False,0.0±0.0,0.0±0.0
2,Biometric Authentication,false_rejection_rate,-3.45,0.064,False,0.0±0.0,3.45±6.64
3,Biometric Authentication,critical_error_rate,-3.03,0.081,False,0.0±0.0,3.03±5.85
4,Biometric Authentication,extraction_error_rate,0.00,NaN,False,0.0±0.0,0.0±0.0
5,Biometric Passive Liveness,missed_fraud_rate,1.88,0.662,False,1.88±1.38,0.0±0.0
6,Biometric Passive Liveness,false_approve_rate,5.13,0.384,False,5.13±1.48,0.0±0.0
7,Biometric Passive Liveness,false_rejection_rate,0.28,0.726,False,0.28±0.17,0.0±0.0
8,Biometric Passive Liveness,critical_error_rate,1.21,0.388,False,1.21±0.32,0.0±0.0
9,Biometric Passive Liveness,extraction_error_rate,0.00,NaN,False,0.0±0.0,0.0±0.0


In [181]:
a = results_df[results_df['Significant']== True].sort_values(by='Delta', ascending=True)
a[['Group', 'Metric', 'Delta, %', 'P-Value',  'Significant', 'Rate_has_video_True, %', 'Rate_has_video_False, %']]

,Group,Metric,"Delta, %",P-Value,Significant,"Rate_has_video_True, %","Rate_has_video_False, %"
19,IDV,extraction_error_rate,1.33,0.001,True,3.47±0.56,2.14±0.5


In [183]:
df.groupby(['primary_product', 'has_video']).agg({'qa_reviewed': 'sum'})

qa_reviewed
primary_product            has_video             
Biometric Authentication   False               33
                           True               100
Biometric Passive Liveness False               61
                           True              4480
Document Verification      False            12346
                           True               932
IDV                        False             3225
                           True              4154